In [1]:
import json, requests
import os
import numpy as np
import pandas as pd

In [2]:
API_KEY = 'JRLUCAS'

In [6]:
def get_price_history(symbols):
    candle_list = []
    df_dict = {}
    for symbol in symbols:
        API_URL = r'https://api.tdameritrade.com/v1/marketdata/{}/pricehistory'.format(symbol)
        params = {
            'apikey': API_KEY,
            'periodType': 'year',
            'period': 2,
            'frequencyType': 'daily',
            'frequency': 1,
            'needExtendedHoursData': 'false'
        }

        content = requests.get(url=(API_URL),params=params).text

        data = json.loads(content)
        candle_list.append(data)
        for i in candle_list:
            df = pd.DataFrame.from_dict(i['candles'])
            pickle_name = symbol
            pickle_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), 'data', pickle_name + '.pickle'))
            df.to_pickle(pickle_path)
    return df

In [7]:
etf_list = ['VTI','VEA', 'VWO', 'VNQ', 'XLE', 'BND', 'SCHP', 'VTEB', 'VIG']
etf_list2 = ['AAPL','CSCO','IBM','AMZN']

In [8]:
get_price_history(etf_list2)

,close,datetime,high,low,open,volume
0,1003.00,1496725200000,1016.5000,1001.2500,1012.0000,3346432
1,1010.07,1496811600000,1010.2490,1002.0000,1005.9500,2823041
2,1010.27,1496898000000,1013.6100,1006.1100,1012.0600,2767857
3,978.31,1496984400000,1012.9900,927.0000,1012.5000,7647692
4,964.91,1497243600000,975.9500,945.0000,967.0000,9447233
5,980.79,1497330000000,984.5000,966.1000,977.9900,4580011
6,976.47,1497416400000,990.3400,966.7100,988.5900,3974900
7,964.17,1497502800000,965.7300,950.8600,958.7000,5373865
8,987.71,1497589200000,999.7500,982.0000,996.0000,11472662
9,995.17,1497848400000,1017.0000,989.9000,1017.0000,5043408
